In [1]:
library(tidyverse)
library(RPostgreSQL)
library(tableone)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
data_dir <- file.path("..", "data")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
 [3] "drugs.sql"               "icd9.sql"               
 [5] "lab_tests.sql"           "lab_unpivot.sql"        
 [7] "merge_data.sql"          "population.sql"         
 [9] "vital_signs.sql"         "vital_signs_unpivot.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
full_data <- dbGetQuery(con, "select * from merged_data")

In [6]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [7]:
names(full_data)

[1] "icustay_id"                     "hadm_id"                       
  [3] "subject_id"                     "first_careunit"                
  [5] "intime"                         "outtime"                       
  [7] "angus"                          "age"                           
  [9] "icu_order"                      "echo_time"                     
 [11] "echo"                           "gender"                        
 [13] "weight"                         "saps"                          
 [15] "sofa"                           "elix_score"                    
 [17] "vent"                           "vaso"                          
 [19] "icu_adm_weekday"                "icu_adm_hour"                  
 [21] "mort_28_day"                    "icu_los_day"                   
 [23] "icd_chf"                        "icd_afib"                      
 [25] "icd_renal"                      "icd_liver"                     
 [27] "icd_copd"                       "icd_cad"                       
 [29] "icd_stroke"                     "icd_malignancy"                
 [31] "vs_temp_flag"                   "vs_temp_first"                 
 [33] "vs_temp_min"                    "vs_temp_max"                   
 [35] "vs_map_flag"                    "vs_map_first"                  
 [37] "vs_map_min"                     "vs_map_max"                    
 [39] "vs_cvp_flag"                    "vs_cvp_first"                  
 [41] "vs_cvp_min"                     "vs_cvp_max"                    
 [43] "vs_heart_rate_flag"             "vs_heart_rate_first"           
 [45] "vs_heart_rate_min"              "vs_heart_rate_max"             
 [47] "lab_hemoglobin_flag"            "lab_hemoglobin_first"          
 [49] "lab_hemoglobin_min"             "lab_hemoglobin_max"            
 [51] "lab_hemoglobin_abnormal"        "lab_wbc_flag"                  
 [53] "lab_wbc_first"                  "lab_wbc_min"                   
 [55] "lab_wbc_max"                    "lab_wbc_abnormal"              
 [57] "lab_lactate_flag"               "lab_lactate_first"             
 [59] "lab_lactate_min"                "lab_lactate_max"               
 [61] "lab_lactate_abnormal"           "lab_po2_flag"                  
 [63] "lab_po2_first"                  "lab_po2_min"                   
 [65] "lab_po2_max"                    "lab_po2_abnormal"              
 [67] "lab_troponin_flag"              "lab_troponin_first"            
 [69] "lab_troponin_min"               "lab_troponin_max"              
 [71] "lab_troponin_abnormal"          "lab_potassium_flag"            
 [73] "lab_potassium_first"            "lab_potassium_min"             
 [75] "lab_potassium_max"              "lab_potassium_abnormal"        
 [77] "lab_creatinine_kinase_flag"     "lab_creatinine_kinase_first"   
 [79] "lab_creatinine_kinase_min"      "lab_creatinine_kinase_max"     
 [81] "lab_creatinine_kinase_abnormal" "lab_creatinine_flag"           
 [83] "lab_creatinine_first"           "lab_creatinine_min"            
 [85] "lab_creatinine_max"             "lab_creatinine_abnormal"       
 [87] "lab_pco2_flag"                  "lab_pco2_first"                
 [89] "lab_pco2_min"                   "lab_pco2_max"                  
 [91] "lab_pco2_abnormal"              "lab_bnp_flag"                  
 [93] "lab_bnp_first"                  "lab_bnp_min"                   
 [95] "lab_bnp_max"                    "lab_bnp_abnormal"              
 [97] "lab_bicarbonate_flag"           "lab_bicarbonate_first"         
 [99] "lab_bicarbonate_min"            "lab_bicarbonate_max"           
[101] "lab_bicarbonate_abnormal"       "lab_bun_flag"                  
[103] "lab_bun_first"                  "lab_bun_min"                   
[105] "lab_bun_max"                    "lab_bun_abnormal"              
[107] "lab_platelet_flag"              "lab_platelet_first"            
[109] "lab_platelet_min"               "lab_platelet_max"              
[111] "lab_platelet_abnormal"          "la

In [8]:
tab_basics <- CreateTableOne(vars = c("age", "gender", "weight", "elix_score",
                                      "saps", "sofa", "vent", "vaso", "sedative",
                                      "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                      "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                      "icu_adm_weekday"),
                             strata = "echo",
                             factorVars = c("gender", "vent", "vaso", "sedative",
                                            "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                            "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                            "icu_adm_weekday"),
                             data = full_data) %>%
    print

                        Stratified by echo
                         0             1             p      test
  n                       3099          3063                    
  age (mean (sd))        84.18 (67.14) 79.17 (59.72)  0.002     
  gender = M (%)          1524 (49.2)   1603 (52.3)   0.014     
  weight (mean (sd))     78.56 (23.58) 82.98 (26.70) <0.001     
  elix_score (mean (sd))  8.51 (7.45)  10.05 (7.68)  <0.001     
  saps (mean (sd))       19.63 (5.79)  20.78 (5.45)  <0.001     
  sofa (mean (sd))        5.30 (3.62)   6.32 (3.80)  <0.001     
  vent = 1 (%)            1459 (47.1)   1808 (59.0)  <0.001     
  vaso = 1 (%)             839 (27.1)   1176 (38.4)  <0.001     
  sedative = 1 (%)        1247 (40.2)   1540 (50.3)  <0.001     
  icd_chf = 1 (%)          563 (18.2)   1198 (39.1)  <0.001     
  icd_afib = 1 (%)         622 (20.1)    988 (32.3)  <0.001     
  icd_renal = 1 (%)        436 (14.1)    503 (16.4)   0.011     
  icd_liver = 1 (%)        314 (10.1)    340 (1

In [9]:
tab_features <- CreateTableOne(vars = c("vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
                                        "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
                                        "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
                                        "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
                                        "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
                                        "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               strata = "echo",
                               factorVars = c("lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               data = full_data) %>%
    print

                                    Stratified by echo
                                     0               1                p     
  n                                    3099            3063                 
  vs_map_first (mean (sd))            79.63 (19.44)  119.76 (2183.54)  0.309
  vs_heart_rate_first (mean (sd))     92.97 (19.87)   95.21 (26.70)   <0.001
  vs_temp_first (mean (sd))           36.75 (1.05)    36.83 (1.58)     0.014
  vs_cvp_first (mean (sd))            11.88 (17.09)   13.57 (19.12)    0.034
  lab_wbc_first (mean (sd))           13.48 (14.03)   13.91 (12.58)    0.215
  lab_hemoglobin_first (mean (sd))    10.55 (1.98)    10.65 (2.04)     0.062
  lab_platelet_first (mean (sd))     223.96 (134.79) 213.44 (127.53)   0.002
  lab_sodium_first (mean (sd))       139.09 (6.45)   138.62 (5.92)     0.003
  lab_potassium_first (mean (sd))      4.10 (0.79)     4.15 (0.85)     0.024
  lab_bicarbonate_first (mean (sd))   22.28 (5.40)    22.26 (5.71)     0.864
  lab_chloride_first 

In [10]:
write.csv(tab_basics, file.path(data_dir, "basics_tbone.csv"))
write.csv(tab_features, file.path(data_dir, "features_tbone.csv"))